# PyTorch Tutorial 14 - Convolutional Neural Network (CNN)

https://www.youtube.com/watch?v=pDdP0TFzsoQ&list=PLqnslRFeH2UrcDBWF5mfPGpqQDSta6VK4&index=14

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

## device configuration

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## hyper parameters

In [3]:
num_epochs = 10
batch_size = 4
learning_rate = 0.001

#### dataset has PILImage images of range [0, 1]; we transform them to Tensors of normalized range [-1, 1]

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

## downloading datasets

In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


## DataLoader

In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## naming classes

In [7]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## implementing convolutional net

In [8]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 3 - color channels, 6 - output size (is up to us), 5 - kernel size
        self.pool = nn.MaxPool2d(2, 2)  # 2 - kernel size, 2 - stride
        self.conv2 = nn.Conv2d(6, 16, 5)  # 6 - input size (=output in conv1), 16 - output size (is up to us), 5 - kernel size
        self.fc1 = nn.Linear(16*5*5, 120)  # fc - fully connected layer, 16*5*5 - final size of an object 
                                           # (see L14_Convolutional_Neural_Network_(CNN)_size_evolution_demonstration)
                                           # 120 - output size (is up to us)
        self.fc2 = nn.Linear(120, 84)  # 120 = fc1 output size, 84 - output size (is up to us)
        self.fc3 = nn.Linear(84, 10)  # 84 = fc2 output size, 10 - because we have 10 classes
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
model = ConvNet().to(device)

## setting loss and optimizer

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  # Stochastic Gradient Descent to optimize model parameters

## training loop

In [11]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: 4, 3, 32, 32 --> 4, 3, 1024
        # input layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        loss.backward()
        
        # optimization
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1) % 2000 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss {loss.item():.4f}')
                  
print('finished training')

epoch 1/10, step 2000/12500, loss 2.3158
epoch 1/10, step 4000/12500, loss 2.2919
epoch 1/10, step 6000/12500, loss 2.2841
epoch 1/10, step 8000/12500, loss 2.2981
epoch 1/10, step 10000/12500, loss 2.3093
epoch 1/10, step 12000/12500, loss 2.5573
epoch 2/10, step 2000/12500, loss 1.7068
epoch 2/10, step 4000/12500, loss 1.8754
epoch 2/10, step 6000/12500, loss 1.9069
epoch 2/10, step 8000/12500, loss 1.6251
epoch 2/10, step 10000/12500, loss 1.6674
epoch 2/10, step 12000/12500, loss 1.3569
epoch 3/10, step 2000/12500, loss 1.2856
epoch 3/10, step 4000/12500, loss 1.2551
epoch 3/10, step 6000/12500, loss 2.1059
epoch 3/10, step 8000/12500, loss 1.2323
epoch 3/10, step 10000/12500, loss 1.8314
epoch 3/10, step 12000/12500, loss 1.4957
epoch 4/10, step 2000/12500, loss 1.6153
epoch 4/10, step 4000/12500, loss 0.9922
epoch 4/10, step 6000/12500, loss 1.8456
epoch 4/10, step 8000/12500, loss 1.2534
epoch 4/10, step 10000/12500, loss 1.6652
epoch 4/10, step 12000/12500, loss 1.3113
epoch 5/

## model evaluation

In [12]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for _ in range(10)]
    n_class_samples = [0 for _ in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value, index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            
    acc = 100. * n_correct / n_samples
    print(f'accuracy of the network: {acc} %')
    
    for i in range(10):
        acc = 100. * n_class_correct[i] / n_class_samples[i]
        print(f'accuracy of {classes[i]}: {acc} %')

accuracy of the network: 58.48 %
accuracy of plane: 69.9 %
accuracy of car: 73.0 %
accuracy of bird: 36.8 %
accuracy of cat: 40.7 %
accuracy of deer: 43.0 %
accuracy of dog: 52.5 %
accuracy of frog: 73.3 %
accuracy of horse: 66.2 %
accuracy of ship: 64.2 %
accuracy of truck: 65.2 %


the model was trained with 10 epochs

#### 4 epochs:

    accuracy of the network: 45.69 %
    accuracy of plane: 47.1 %
    accuracy of car: 51.0 %
    accuracy of bird: 20.9 %
    accuracy of cat: 13.6 %
    accuracy of deer: 36.5 %
    accuracy of dog: 45.4 %
    accuracy of frog: 67.1 %
    accuracy of horse: 55.3 %
    accuracy of ship: 56.9 %
    accuracy of truck: 63.1 %